In [189]:
Image(url='http://files.obshandinhand.webnode.nl/200000026-f2e8cf3e1e/200000000.jpg')

#Mine the Gap: Targeting High Risk Patients for Care Coordination & Social Support Services

*This notebook originally appeared as a [post](http://jakevdp.github.io/blog/2014/11/11/the-hipster-effect-interactive/) on the blog [Pythonic Perambulations](http://jakevdp.github.io). The content is BSD licensed.*

This week I started seeing references all over the internet to [this paper](http://arxiv.org/abs/1410.8001): *The Hipster Effect: When Anticonformists All Look The Same*. It essentially describes a simple mathematical model which models conformity and non-conformity among a mutually interacting population, and finds some interesting results: namely, **conformity among a population of self-conscious non-conformists is similar to a phase transition in a time-delayed thermodynamic system**. In other words, with enough hipsters around responding to delayed fashion trends, a plethora of facial hair and fixed gear bikes is a natural result.

Also naturally, upon reading the paper I wanted to try to reproduce the work. The paper solves the problem analytically for a continuous system and shows the precise values of certain phase transitions within the long-term limit of the postulated system. Though such theoretical derivations are useful, I often find it more intuitive to simulate systems like this in a more approximate manner to gain hands-on understanding. By the end of this notebook, we'll be using IPython's incredible [interactive widgets](http://nbviewer.ipython.org/github/ipython/ipython/blob/master/examples/Interactive%20Widgets/Index.ipynb) to explore how the inputs to this model affect the results.

___

Contents:

* [Think Python](http://www.greenteapress.com/thinkpython/thinkpython.pdf)
* [MIT Open Courseware: A Gentle Introduction to Programming Using Python](http://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-189-a-gentle-introduction-to-programming-using-python-january-iap-2008/index.htm)
* [Learn Python the Hard Way](http://learnpythonthehardway.org/book/)
* [Python Koans](https://github.com/gregmalcolm/python_koans/wiki)

___

In [134]:
# Modified Stylesheet for notebook.
# from IPython.core.display import HTML
# def css_styling():
#     styles = open("../custom_nb_style.css", "r").read()
#     return HTML(styles)

# css_styling()

In [188]:
#imports
# import the Image display module
from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

## ___Project Overview___

In [136]:
Image(url='http://www.cambridgeshireinsight.org.uk/files/caminsight/resize/widerdeterminants-500x337.jpg', height=50)

In [137]:
#project overview text

In [138]:
#project overview viz

In [139]:
#project overview viz text

### Problem

> ###From an at-risk patient's perspective, where are the gaps in care?
> ###Ultimately, where are the opportunities for improved system management for most at-risk enrollees?

<a id="goals"></a>
###Goal

Let's ask further questions to better define what we hope to achieve.

> ####Exploratory Analysis

> What factors are associated with poor and good health status?

> Are there any general patterns in comorbidity?

> What is the impact of pain on physical and mental quality of living?

> What is the relationship between mental health and presence of multiple chronic conditions?

> Regional Differences?

> ####Cohort Analysis

> Among Medicare enrollees, which patients are among the most at-risk?

> What are there corresponding demographic and outcomes profiles on total health dimensions?

> How does an individual enrollee's profile compare to Medicaid population?

> ####Predictive Modeling

> Can we approximate/predict a new enrollee's profile from historical data?

##___Acquire___

The data comes from an HOS Outcomes Survey from 2012. 

In [140]:
#graphic of data dictionary - pre clean

In [141]:
#graphic of data dictionary - post clean

In [159]:
import pandas as pd
import numpy as np

####Raw data from [hosonline.org](hosonline.org):

In [2]:
!head -n 1 source_data/C16B_PUF.csv

#### The Final Dataset

original data major components

The dataset consists of XXXX elements comprised of XXXX unique songs from XXXX artists, spanning   XXX weeks.

In [ ]:
# songs = pd.read_csv('gist')
# songs.head()

##___Explore Data___

In [ ]:
pd.scatter_matrix(songs[['popularity','danceability','duration','tempo','loudness','energy']],figsize=(20,20))
plt.suptitle('Scatterplot Matrix of Song Attributes',size=25)

In [ ]:
rock = rock.dropna()
plt.figure(figsize=(20,20))
for i in range(len(features)):
    v = i +1
    ax1 = subplot(5,2,v)
    ax1.hist(rock[features[i]])
    ax1.set_title(str(features[i]),fontsize=15)

In [5]:
#explanatory text of data attributes: mean, variance spread

Despite the randomness of the scatterplots, the histograms of the features were almost normally distributed.  I found the distribution of the data to be surprising considering the weekly top 40 is a seemingly random collection of songs with no real relation to one another.

Furthermore, the scatterplots indicated that there would be a lot of noise in the data, given the wide range of song attributes.  The way to analyze the data, I determined, would be to use subsets.  

In [ ]:
print "Plotting Large Correlation Matrix"

f, ax = plt.subplots(figsize=(9, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.corrplot(profiles_simple, annot=False, sig_stars=False,
             diag_names=False, cmap=cmap, ax=ax)
f.tight_layout()

___

## ___Initial Analysis - Cluster Analysis___

In [6]:
#Text How would we visually segment?  We will compare this later to clustering technique. 

In [ ]:
#Viz

###Building Model

show math:
    
$$
\phi(x;\beta) = \frac{1 + \tanh(\beta \cdot x)}{2}
$$

In [8]:
#Text

To train and score my model, I used a train/test split, which, as the name implies, breaks the dataset up into a training set and a testing set.  The training set is fed into the model to 'teach' the feature variations and their relationship to the target variable and the test set is used to simulate new data that the computer has not seen.  The computer will then score how well the model performs.  In the case of the OLS model, the computer will provide R-Squared values. 

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances

km = KMeans(n_clusters=5, random_state=1)
km.fit(profiles_simple)

In [ ]:
profiles_simple['persona'] = km.predict(profiles_simple)

In [ ]:
profiles_simple.groupby(by='persona').mean()

In [ ]:
profiles_simple[profiles_simple.persona == 4].hist(figsize=(12, 8));

In [ ]:
profiles_simple[profiles_simple.persona == 2].hist(figsize=(12, 8));

In [ ]:
profiles_simple[profiles_simple.persona == 1].hist(figsize=(12, 8));

#### Some things to note about our goals and approach:

* Determine what deﬁnes __success__, and to what degree.
* Brainstorm __metrics__ to visualize and/or calculate.
* Ask __questions__ that have (or can have) a definitive answer.
* Be careful what you wish for, be aware of possible __correlations__, and take caution with how you [measure](http://en.wikipedia.org/wiki/Observer-expectancy_effect) it.

###Scoring Model

In [12]:
# print 'Train Score: ', model.score(xtrain,ytrain)
# print 'Test Score: ', model.score(xtest,ytest)

Both the training and test scores indicate a relatively poor linear fit for the metadata features and elapsed time vs the target value of popularity.  As expected, the model does not show signs of overfitting as the test and train score values are nearly identical.  

###Examing Model & Coefficients

####Feature Analysis

In [ ]:
#four groups boxplots plotted next to each other.
plt.figure(figsize=(20,20))
for i in range(len(features)):
    v = i +1
    ax1 = rock.boxplot(features[i],by='decade',ax=subplot(5,2,v))
    ax1.set_title(str(features[i]),fontsize=15)
    ax1.set_xlabel('')
plt.suptitle('Rock Feature Boxplots by Decade',size=20)

In [ ]:
#compare this clustering to just using quartiles.

According to the boxplots, songs in the top median quantile of popularity demonstrate a slightly higher level of verbosity, energy, and loudness than songs in the lower median quantile.

In [ ]:
#Pros & Cons of other models

Upon examination of the training set, the target values are skewed towards the earlier decades in the dataset.  With the high variation in the data, a Naïve Bayes model, although a strong algorithm, will not pick up on the nuances in the song metadata to make effective predictions to the song’s decade of origin.  This is because Naive Bayes assumes that all features are independent of one another.

In place of Naïve Bayes, I will use a Random Forest model.  Although very prone to overfitting on the training set in machine learning models, this model is strong enough to pick up on the subtle variations in the data, as it does not perceive all of the features to be in a vacuum like the Bayes algorithm.  Moreover, as the rock song dataset only consists of 5,056 unique songs, scaling with hundreds decision trees should not prove to be an issue.


___

##Conclusions

Through the use of song metadata for the Billboard Top 40 for the past half century and OLS, Logistic, Random Forest, and K-Means machine learning algorithms, I determined that time plays a larger role in the popularity attrition of a song than any metadata factor in an OLS linear model.  Using a categorial logistic regression model, however, I found that loudness, energy, and danceability, to some extent, account for some of the gains in song popularity. 

Moreover, depending on the strength of the algorithm used, song metadata can be a decent predictor to determine from which decade a particular song comes.  The metadata truly shines, however, in the classification and clustering of genres.  In developing my own genre clusters with K-Means, I was amazed how accurately the computer picked like-sounding songs through only 5 feature attributes.  

As the genre area plot demonstrated, we as a culture are getting more diverse in terms of the music to which we listen.  With the power of machine learning, however, services like Spotify, Google Music, and Pandora will never be at a loss for songs to meet a particular individual’s tastes, despite the ever increasing scope of possibilities.


___

##Further Research

___

### More Information

For more information please visit:

For more code and the datasets for this project, please visit my GitHub page:

In [ ]:
# from IPython.core.display import HTML
# styles = open("custom_dark.css", "r").read()
# HTML(styles)

##### For prettier graphs see http://localhost:8888/notebooks/project/iPython%20notebook%20gallery/Up%20and%20Down%20PyData%202014.ipynb